In [ ]:
from fastai.vision import *

import pandas as pd 
import numpy as np 
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

proj_path = r"dataset"
p_path = Path(proj_path)

df = pd.read_csv("modified_category.csv")
df.head()

#PICK IMAGE SIZE#
img_size = 256

np.random.seed(90)
data = (ImageList.from_df(df, "./")
                    .split_by_rand_pct(0.2)
                    .label_from_df()
                    .transform(get_transforms(), size= img_size)
                    .databunch(bs=6)).normalize(imagenet_stats)

data.show_batch()

In [ ]:
############ Create Model ############
from fastai.metrics import accuracy, Precision, Recall
learn = cnn_learner(data, models.resnet34, metrics=[accuracy, Precision(average='macro'), Recall(average='macro')], opt_func=optim.SGD)

#categories to predict
data.classes

#no of categories present
data.c

#length of training set
len(data.train_ds)

#length of validation set
len(data.valid_ds)

In [ ]:
############ Train Model ############
# Build the CNN model with the pretrained resnet34
# Error rate = 1 - accuracy

#EDIT NUMBER OF EPOCHS, CAN ALSO ADD A LEARNING RATE (e.g. lr = 0.003)    
# Train the model on # epochs of data at the default learning rate
learn.data.batch_size = 50
learn.unfreeze()

In [ ]:
learn.fit(10, 0.005)


#NOT SURE WHERE/HOW IT'S SAVING IT
# save model
learn.save('res34-stage1')

############ Evaluate Model ############
interp = ClassificationInterpretation.from_learner(learn)

# confusion matrix: actual on left, predicted on right
interp.plot_confusion_matrix(figsize=(17, 17))


# print out the most mis-classified classes
interp.most_confused()

# plotting the top losses
interp.plot_top_losses(6, figsize=(25,25))